In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# getting url list to feed into extraction
links = pd.DataFrame(pd.read_csv('data/intern_train.csv'))
links_list = list(links.url)

In [3]:
test_links = pd.DataFrame(pd.read_csv("data/intern_test.csv"))
test_links_list = list(test_links.url)

In [4]:
test_links.label[-7:]

196    1
197    0
198    0
199    0
200    0
201    0
202    0
Name: label, dtype: int64

# Newspaper3k Extraction

In [13]:
from newspaper import fulltext

def get_text(url):
    
    # getting section and location parts (if it exists) from links 
    meta = []
    for part in url.split('/'):
        
        if part.startswith('tp-'):
            meta.append(part[3:])
        
    section = "None"
    location = "None"
    
    if len(meta) == 2:
        section = meta[0]
        location = meta[1]
    elif len(meta) == 1:
        section = meta[0]  
    elif len(meta) == 0:
        pass
    
    # getting the htmls
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    
    # extract the headline
    headline = soup.find_all('h1')[0].text.strip()
    
    # get the text
    text = fulltext(html).replace('\n',' ').replace('  ',' ')
    
    
    
    # clean the staff reporter part. usually its the beginning of the news.
    if text.startswith("Staff Reporter"):
        text = text[text.index(":")+2:]
    
    
    return url,section,location,headline,text

In [46]:
def create_documents(link_df):
    # this function will take the list of the urls  and extract one by one and save it into a list which will be a 
         # dataframe later on. and saving it as .csv to use it. 
        
    l_l = list(link_df.url)
    results = []
    
    for url in l_l:
        idx =l_l.index(url)
        label = int(list(link_df.label)[idx:idx+1][0])

        r = requests.get(url)
        
        
        url,section,location,headline,text=get_text(url)
        
        results.append([url,section,location,headline,text,label])
        
    return results

In [47]:
res_test = create_documents(test_links)

In [48]:
test_df = pd.DataFrame(res_test)

In [51]:
test_df.to_csv("data/extracted_test_data.csv")

In [36]:
res = create_documents(links_list)

In [37]:
pd.DataFrame(res).to_csv("data/extracted_data.csv")